Defines and imports

In [ ]:
import json
import statistics
from collections import defaultdict
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots

paths = {
    "robocup": "../../dtn7-lab/shared/scenarios/robocup/",
    "robocup_base": "../../dtn7-lab/shared/scenarios/robocup_baseline/",
    "industrial": "../../dtn7-lab/shared/scenarios/industrial/",
    "industrial_baseline": "../../dtn7-lab/shared/scenarios/industrial_baseline/"
    }

stats = defaultdict(list)

for name, path in paths.items():
    for root, _, files in os.walk(path):
        for file in files:
            if file == "stats.json":
                with open(f"{root}/{file}", "r") as f:
                    stats[name].append(json.load(f))

# colors = ["#ab63fa", "#00cc96", "#ffa15a"]
colors = ["rgba(171, 99, 250, 1)", "rgba(0, 204, 150, 1)", "rgba(255, 161, 90, 1)"]

def colorAlpha(colorStr):
    return colorStr.replace(", 1", ", 0.3")

def mean(data):
    n = 0
    mean = 0.0
 
    for x in data:
        n += 1
        mean += (x - mean)/n

    if n < 1:
        return float('nan')
    else:
        return mean


Helper

In [ ]:
def overTimeWithStdev(fig, colorCnt, x, y, upper, lower, name, yAxis = None):
    global colors
    fig.add_trace(
        go.Scatter(
            x=x,
            y=y,
            yaxis=yAxis,
            mode="lines",
            legendgroup=colorCnt,
            line=dict(color=colors[colorCnt], width=2),
            name=name,
        ),
    )

    fig.add_trace(
        go.Scatter(
            x=x + x[::-1],
            y=upper + lower[::-1],
            yaxis=yAxis,
            fill="tozerox",
            fillcolor=colorAlpha(colors[colorCnt]),
            legendgroup=colorCnt,
            line=dict(color="rgba(0, 0, 0, 0)"),
            showlegend=False,
        ),
    )


DTN Graphs

In [ ]:
def transmissionProbability():
    global stats

    fig = go.Figure()
    fig.update_xaxes(title_text="Time [s]", dtick=120)
    fig.update_yaxes(
        title_text="Delivery Probability",
        # range=[0,1.1]
    )
    fig.update_layout(title_text="Delivery Probability over all Topics")

    colorCnt = 0
    for scenario, localStats in stats.items():
        timestamps = []
        averages = []
        upper = []
        lower = []

        for timestamp in range(4330):
            timestamps.append(timestamp)
            values = []
            for d in localStats:
                value = d["delivery"][str(timestamp)]
                if value is not None:
                    values.append(value)
            if len(values) > 1:
                m = statistics.mean(values)
                d = statistics.stdev(values)
                averages.append(m)
                upper.append(m + d)
                lower.append(m - d)
            else:
                averages.append(None)
                upper.append(None)
                lower.append(None)

        overTimeWithStdev(fig, colorCnt, timestamps, averages, upper, lower, scenario)

        colorCnt += 1

    fig.show()

transmissionProbability()

Bundles in Flight (between base and robots)

In [ ]:
def bundlesInFlight():
    global stats
    global colors

    fig = go.Figure()
    fig.update_xaxes(title_text="Time [s]", dtick=120)
    # fig.update_yaxes(
    #     title_text="Number of bundles in Flight",
    #     secondary_y=False,
    #     titlefont=dict(color=colors[0]),
    #     tickfont=dict(color=colors[0]),
    # )
    # fig.update_yaxes(
    #     title_text="Number of bundles in Flight",
    #     secondary_y=True,
    #     overlaying="y",
    #     tickmode="sync",
    #     titlefont=dict(color=colors[1]),
    #     tickfont=dict(color=colors[1]),
    # )
    fig.update_layout(
        title_text="Bundles in Flight",
        yaxis=dict(
            title=dict(text="Number of bundles in Flight"),
            side="left",
            range=[0, 250],
            overlaying="y2",
            tickmode="sync",
        ),
        yaxis2=dict(
            title=dict(text="Number of bundles in Flight"),
            side="right",
            range=[0, 20000],
        ),
    )

    colorCnt = 0
    for scenario, localStats in stats.items():
        timestamps = []
        averages = []
        upper = []
        lower = []

        for timestamp in range(4330):
            timestamps.append(timestamp)
            values = []
            for experiment in localStats:
                value = experiment["bundlesInFlight"][timestamp]
                if value is not None:
                    values.append(value)
            if len(values) > 1:
                m = statistics.mean(values)
                d = statistics.stdev(values)
                averages.append(m)
                upper.append(m + d)
                lower.append(m - d)
            else:
                averages.append(None)
                upper.append(None)
                lower.append(None)

        if "base" in scenario:
            overTimeWithStdev(
                fig, colorCnt, timestamps, averages, upper, lower, scenario, "y2"
            )
        else:
            overTimeWithStdev(
                fig, colorCnt, timestamps, averages, upper, lower, scenario
            )

        colorCnt += 1

    fig.show()


bundlesInFlight()


E2E Latency

In [ ]:
def e2eLatency():
    global stats

    for scenario, scenarioStats in stats.items():
        fig = go.Figure()
        fig.update_yaxes(
            title_text="Latency [s]",
        )
        fig.update_layout(
            title_text=f"End-to-End Latency per Topic - {scenario}", height=600
        )

        valuesPerTopicConnected = defaultdict(list)
        valuesPerTopicDisconnected = defaultdict(list)
        for experiment in scenarioStats:
            for topic, values in experiment["e2e"].items():
                for value in values:
                    # TODO: check thresold
                    if value < 1:
                        valuesPerTopicConnected[topic].append(value)
                    else:
                        valuesPerTopicDisconnected[topic].append(value)

        # for topic, values in valuesPerTopicConnected.items():
        for topic, values in valuesPerTopicDisconnected.items():
            fig.add_trace(
                go.Violin(
                    y=values,
                    name=topic,
                    box_visible=True,
                    width=0.8,
                    meanline_visible=True,
                    # boxpoints=False,
                    # points="all",
                )
            )

        fig.show()


e2eLatency()


Utilization

In [ ]:
def utilizationBundles():
    global stats

    fig = make_subplots(
        rows=2,
        cols=2,
        subplot_titles=("robocup", "robocup_base", "industrial", "industrial_baseline"),
    )
    fig.update_yaxes(
        title_text="Number of bundles",
    )
    fig.update_layout(title_text=f"Overall Transmitted Bundles", showlegend=False, height=800)

    rowCnt = 1
    colCnt = 1
    for scenario, scenarioStats in stats.items():
        if colCnt > 2:
            colCnt = 1
            rowCnt += 1
        counts = []
        for experiment in scenarioStats:
            counts.append(experiment["bundleCount"])

        fig.add_trace(
            go.Box(
                y=counts,
                name=scenario,
                boxmean=True,
                boxpoints=False,
                # boxpoints="outliers",
            ),
            row=rowCnt,
            col=colCnt,
        )

        colCnt += 1

    fig.show()


def utilizationTraffic():
    global stats

    fig = make_subplots(
        rows=2,
        cols=2,
        subplot_titles=("robocup", "robocup_base", "industrial", "industrial_baseline"),
    )
    fig.update_yaxes(
        title_text="Network Traffic [B]",
    )
    fig.update_layout(title_text=f"Total Network Traffic", showlegend=False, height=800)

    rowCnt = 1
    colCnt = 1
    for scenario, scenarioStats in stats.items():
        if colCnt > 2:
            colCnt = 1
            rowCnt += 1
        counts = []
        for experiment in scenarioStats:
            counts.append(experiment["totalTraffic"])

        fig.add_trace(
            go.Box(
                y=counts,
                name=scenario,
                boxmean=True,
                boxpoints=False,
                # boxpoints="all",
            ),
            row=rowCnt,
            col=colCnt,
        )

        colCnt += 1

    fig.show()


utilizationBundles()
utilizationTraffic()
